In [5]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [6]:
# Import libraries for data augumentation

import cv2
import os
import numpy as np
import shutil
from skimage import exposure
from PIL import Image
from IPython.display import display
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import random
import time

random.seed(123) # random data augumentation


In [7]:


# this function does the job of image zoom augumentation | input:numpy img outut:zoomed numpy img
def zoom_augmentation(image, zoom_factor):
    # Ensure the zoom factor is within valid range (0.0 to 1.0)
    if zoom_factor <= 0.0 or zoom_factor >= 1.0:
        raise ValueError("Zoom factor must be between 0.0 and 1.0")

    height, width, _ = image.shape

    # Calculate the cropping region
    crop_height = int(height * zoom_factor)
    crop_width = int(width * zoom_factor)

    # Calculate the starting point for cropping
    start_x = (width - crop_width) // 2
    start_y = (height - crop_height) // 2

    # Crop the image to achieve zoom
    zoomed_image = image[start_y:start_y + crop_height, start_x:start_x + crop_width]

    # Resize the cropped region back to the original image size
    zoomed_image = cv2.resize(zoomed_image, (width, height))
    return zoomed_image


#--> This function combines many agumentations and output an augumented image in numpy format
def augment_image(image,
                  rotation_angle=0,
                  horizontal_flip=False,
                  vertical_flip=False,
                  brightness_factor=None,
                  apply_hist_eq=False,
                  zoom_factor=1.0):

    augmented_image= image.copy()

    # Rotation
    if rotation_angle != 0:
        rows, cols, _ = augmented_image.shape
        M = cv2.getRotationMatrix2D((cols / 2, rows / 2), rotation_angle, 1)
        augmented_image = cv2.warpAffine(augmented_image, M, (cols, rows))


    # Horizontal flip
    if horizontal_flip:
        augmented_image = cv2.flip(augmented_image, 1)


    # Vertical flip
    if vertical_flip:
        augmented_image = cv2.flip(augmented_image, 0)


    # Brightness adjustment
    if brightness_factor is not None:
        augmented_image = cv2.convertScaleAbs(augmented_image, alpha=brightness_factor, beta=0)


    # histogram equilization
    if apply_hist_eq:
        augmented_image = exposure.equalize_adapthist(augmented_image, clip_limit=0.008)
        augmented_image = (augmented_image*250).astype('uint8')


    # apply zoom
    if zoom_factor!=1.0: # lower is no. higher is zoom | 0.0 to 1.0
        augmented_image = zoom_augmentation(augmented_image, zoom_factor)


    return augmented_image



# ## example use of image augumentation displaying below
# image = cv2.imread('knee-xray/MedicalExpert-I-modified/test/0Normal/NormalG0 (433).png')
# image = augment_image(image, brightness_factor=1.0, zoom_factor=1.0, rotation_angle=10, apply_hist_eq=True, horizontal_flip=True, vertical_flip=True)
# display(Image.fromarray((image).astype('uint8')))



In [8]:
# --> Function to save augumented images

def auto_augument_img(image, generate_qty):
    augumented_images = []


    # Here is augumentation parameters | modify if needed
    rotation_angle = [ +12, 0, -12]
    brightness_factor = [1.0, 0.8]
    vertical_flip = [False, True]
    horizontal_flip = [True]
    apply_hist_eq = [False] # if change to [True], also uncomment hist eql. for original img (below code)
    zoom_factor = [1.0, 0.9]

    total_possible_img_count = len(rotation_angle)* len(brightness_factor)* len(vertical_flip)* len(horizontal_flip)*len(zoom_factor) # 240
    skip_factor = generate_qty/total_possible_img_count # 14/240 = 0.0058
    for rotatation in rotation_angle:
        for bright in brightness_factor:
            for v_flip in vertical_flip:
                for h_flip in horizontal_flip:
                    for zoom in zoom_factor:
                        if random.random() < skip_factor: # 0.0058 lower is number more images
                            aug_im = augment_image(image, brightness_factor=bright, zoom_factor=zoom, rotation_angle=rotatation, apply_hist_eq=apply_hist_eq, horizontal_flip=h_flip, vertical_flip=v_flip)
                            augumented_images.append(aug_im)


    return augumented_images


# # example use: takes an image_arr and gives list of images after applying different augumentation on them | input and output are np array img
# image = cv2.imread('archive/test/0/9003175L.png')
# all_imgs = auto_augument_img(image, generate_qty=10)
# print('Total no. of images generated: ', len(all_imgs))

## Balancing no. of images with data augmentation

In [9]:
# creating new image dataset with data augumentation


def data_balanceing(output_path, folder_path, max_image):
    os.makedirs(output_path, exist_ok=True)

    classes = ['0', '1', '2', '3', '4']  # class folder names

    for class_ in classes:
        img_folder_path = os.path.join(folder_path, str(class_))
        image_counter = 0
        for img_file in os.listdir(img_folder_path): # loop through every single image in class folder
            full_path_img_file = os.path.join(img_folder_path, img_file)
            output_folder_cls = os.path.join(output_path, str(class_))

            # print(full_path_img_file, output_folder_cls)
            os.makedirs(output_folder_cls, exist_ok=True)

            if image_counter>max_image:
                break # break from this class of image

            total_available_img = len(os.listdir(img_folder_path)) # eg: 200
            required_img = max(max_image-total_available_img, 0) # eg: 1000 - 200 = 800
            required_img = required_img/total_available_img # per img reqire img

            # # Original image apply histogram equilization and saving
            # original_img = exposure.equalize_adapthist(cv2.imread(full_path_img_file), clip_limit=0.008)
            # original_img = (original_img*255).astype('uint8')
            # cv2.imwrite(f'{output_folder_cls}/original_{image_counter}.jpg', original_img)
            cv2.imwrite(f'{output_folder_cls}/original_{image_counter}.jpg', cv2.imread(full_path_img_file))

            image_counter+=1

            all_imgs = auto_augument_img(cv2.imread(full_path_img_file), generate_qty=required_img)
            for img_arr in all_imgs:
                new_img_file_path = f'{output_folder_cls}/aug_{image_counter}.jpg'
                cv2.imwrite(new_img_file_path, img_arr)
                image_counter+=1

            if image_counter>max_image:
                break

        print(f'Done class {class_} for {folder_path}')



# Balanceing Train dataset with augmented images

data_balanceing(
    output_path = "/content/drive/MyDrive/SHSU-Projects/Final_files/KneeOA_Dataset_1_Augmented/Train_Augmented",
    folder_path="/content/drive/MyDrive/SHSU-Projects/Final_files/KneeOA_Dataset_1_Original/train",
    max_image=1400
)

Done class 0 for /content/drive/MyDrive/SHSU-Projects/Final_files/KneeOA_Dataset_1_Original/train
Done class 1 for /content/drive/MyDrive/SHSU-Projects/Final_files/KneeOA_Dataset_1_Original/train
Done class 2 for /content/drive/MyDrive/SHSU-Projects/Final_files/KneeOA_Dataset_1_Original/train
Done class 3 for /content/drive/MyDrive/SHSU-Projects/Final_files/KneeOA_Dataset_1_Original/train
Done class 4 for /content/drive/MyDrive/SHSU-Projects/Final_files/KneeOA_Dataset_1_Original/train


**Balanceing Validation dataset with augmented images**

In [10]:
data_balanceing(
    output_path = "/content/drive/MyDrive/SHSU-Projects/Final_files/KneeOA_Dataset_1_Augmented/Val_Augmented",
    folder_path="/content/drive/MyDrive/SHSU-Projects/Final_files/KneeOA_Dataset_1_Original/val",
    max_image=500
)

Done class 0 for /content/drive/MyDrive/SHSU-Projects/Final_files/KneeOA_Dataset_1_Original/val
Done class 1 for /content/drive/MyDrive/SHSU-Projects/Final_files/KneeOA_Dataset_1_Original/val
Done class 2 for /content/drive/MyDrive/SHSU-Projects/Final_files/KneeOA_Dataset_1_Original/val
Done class 3 for /content/drive/MyDrive/SHSU-Projects/Final_files/KneeOA_Dataset_1_Original/val
Done class 4 for /content/drive/MyDrive/SHSU-Projects/Final_files/KneeOA_Dataset_1_Original/val
